In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/emotions/text.csv


In [ ]:
"""
The model clssifies Text coments to Six categories: sadness (0), joy (1), love (2), anger (3), fear (4), and surprise (5).

https://www.kaggle.com/datasets/nelgiriyewithana/emotions
"""

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords




nltk.download('stopwords')
data = pd.read_csv(r"/kaggle/input/emotions/text.csv")
data.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Unnamed: 0,text,label
0,0,i just feel really helpless and heavy hearted,4
1,1,ive enjoyed being able to slouch about relax a...,0
2,2,i gave up my internship with the dmrg and am f...,4
3,3,i dont know i feel so lost,0
4,4,i am a kindergarten teacher and i am thoroughl...,4


In [3]:
data.drop(['Unnamed: 0'], axis = 1, inplace=True)
data

,text,label
0,i just feel really helpless and heavy hearted,4
1,ive enjoyed being able to slouch about relax a...,0
2,i gave up my internship with the dmrg and am f...,4
3,i dont know i feel so lost,0
4,i am a kindergarten teacher and i am thoroughl...,4
...,...,...
416804,i feel like telling these horny devils to find...,2
416805,i began to realize that when i was feeling agi...,3
416806,i feel very curious be why previous early dawn...,5
416807,i feel that becuase of the tyranical nature of...,3


In [ ]:
# Importing english stop words and preprocessing text
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  
    text = re.sub(r'@\w+|\#', '', text)  
    text = re.sub(r'[^A-Za-z\s]', '', text)  
    text = " ".join(word for word in text.split() if word not in stop_words)
    return text



data['clean_text'] = data['text'].apply(clean_text)

In [5]:
data.drop('text', axis =1, inplace=True)
data.head()

,label,clean_text
0,4,feel really helpless heavy hearted
1,0,ive enjoyed able slouch relax unwind frankly n...
2,4,gave internship dmrg feeling distraught
3,0,dont know feel lost
4,4,kindergarten teacher thoroughly weary job take...


In [ ]:
#splitting
X_train, X_test, y_train, y_test = train_test_split(
    data['clean_text'], data['label'], test_size = 0.3, random_state= 42, stratify =data['label'])

In [ ]:
#vectorizing
tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf= tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
#training
model = LogisticRegression(max_iter=200)
model.fit(X_train_tfidf, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=200)

In [ ]:
#evaluation
y_pred = model.predict(X_test_tfidf)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94     36356
           1       0.92      0.93      0.93     42320
           2       0.81      0.77      0.79     10366
           3       0.90      0.90      0.90     17195
           4       0.85      0.84      0.85     14314
           5       0.77      0.72      0.74      4492

    accuracy                           0.90    125043
   macro avg       0.86      0.85      0.86    125043
weighted avg       0.90      0.90      0.90    125043

